# StrongholdNet: Train an RNN (LSTM) to navigate through a Stronghold

The idea is that we interpret the (shortest) path from any room in the stronghold to the portal room as *sequential data* that we feed to an RNN.

In [77]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras as K
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from dataset_rnn import parse_tree_generator, print_stronghold_tree
from anytree import Node, RenderTree, Walker
from anytree.search import find_by_attr
import random

In [85]:
df = pd.read_csv('100k_dataset_rnn.csv', delimiter=' ')

In [158]:
df.head(24)

,stronghold,room,downwards,orientation,parent_room,parent_exit,child_room_1,child_room_2,child_room_3,child_room_4,child_room_5,exit
0,0,RightTurn,0,S,Stairs,1,Stairs,None,None,None,None,0
1,0,Stairs,0,S,Corridor,2,RightTurn,None,None,None,None,0
2,0,Corridor,0,W,FiveWayCrossing,1,Corridor,Stairs,RightTurn,None,None,1
3,0,Corridor,1,W,Corridor,1,LeftTurn,Corridor,None,None,None,2
4,0,Corridor,1,S,Corridor,2,Corridor,None,None,None,None,1
5,0,Corridor,1,S,Corridor,1,RightTurn,None,None,None,None,1
6,0,RightTurn,1,S,Corridor,1,PortalRoom,None,None,None,None,1
7,1,PrisonHall,0,N,SquareRoom,1,Corridor,None,None,None,None,0
8,1,SquareRoom,0,N,ChestCorridor,1,PrisonHall,RightTurn,ChestCorridor,None,None,0
9,1,ChestCorridor,0,N,RightTurn,1,SquareRoom,None,None,None,None,0


Let's apply a custom one-hot encoding…

In [121]:
def onehot(df: pd.DataFrame):
    room_to_vector = {
            'Corridor': 0,
            'PrisonHall': 1,
            'LeftTurn': 2,
            'RightTurn': 3,
            'SquareRoom': 4,
            'Stairs': 5,
            'SpiralStaircase': 6,
            'FiveWayCrossing': 7,
            'ChestCorridor': 8,
            'Library': 9,
            'PortalRoom': 10,
            'SmallCorridor': 11,
            'Start': 12,
            'None': 13}
    orientation_to_vector = {
            'N': 0,
            'S': 1,
            'E': 2,
            'W': 3}
    room_columns = [
            'room',
            'parent_room',
            'child_room_1',
            'child_room_2',
            'child_room_3',
            'child_room_4',
            'child_room_5']
    exit_columns = [
            'parent_exit']
    orientation_columns = [
            'orientation']
    n_exits = 6
    
    df_ = df.copy()

    # one-hot rooms
    for c in room_columns:
        for v in room_to_vector:
            df_[c] = df_[c].replace(v, room_to_vector[v])
        onehot = pd.DataFrame(
                K.utils.to_categorical(df_[c], num_classes=len(room_to_vector)),
                columns=[c + '_is_' + j for j in room_to_vector])
        df_ = pd.concat([onehot, df_], axis=1)

    # one-hot exits
    for c in exit_columns:
        onehot = pd.DataFrame(
                K.utils.to_categorical(df_[c], num_classes=n_exits),
                columns=[c + '_is_' + str(j) for j in range(n_exits)])
        df_ = pd.concat([onehot, df_], axis=1)

    # one-hot orientations
    for c in orientation_columns:
        for v in orientation_to_vector:
            df_[c] = df_[c].replace(v, orientation_to_vector[v])
        onehot = pd.DataFrame(
                K.utils.to_categorical(df_[c], num_classes=len(orientation_to_vector)),
                columns=[c + '_is_' + j for j in orientation_to_vector])
        df_ = pd.concat([onehot, df_], axis=1)
            
            
    df_.drop([k for k in room_columns + exit_columns + orientation_columns], axis=1, inplace=True)
    return df_

In [122]:
df_onehot = onehot(df)

In [124]:
df_onehot.head()

,orientation_is_N,orientation_is_S,orientation_is_E,orientation_is_W,parent_exit_is_0,parent_exit_is_1,parent_exit_is_2,parent_exit_is_3,parent_exit_is_4,parent_exit_is_5,...,room_is_FiveWayCrossing,room_is_ChestCorridor,room_is_Library,room_is_PortalRoom,room_is_SmallCorridor,room_is_Start,room_is_None,stronghold,downwards,exit
0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
1,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
2,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,2
4,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,1


We need to group our sequences by stronghold.

In [126]:
def sequencelify(df):
    for window in range(1, 10):
        for _, s in df.groupby('stronghold'):
            # each stronghold is a batch
            X_batch = []
            y_batch = []
            s = s.drop('stronghold', axis=1).to_numpy()
            for t in range(s.shape[0] - window + 1):
                if s.shape[0] < window:
                    continue
                X = s[t:t+window, :-1]
                X = X.reshape(window, X.shape[1])
                y = np.array(s[t+window-1, -1])
                X_batch.append(X)
                y_batch.append(y)
            X_batch = np.array(X_batch)
            y_batch = np.array(y_batch)
            X_batch = X_batch.reshape(s.shape[0] - window + 1, window, s.shape[1] - 1)
            y_batch = y_batch.reshape(s.shape[0] - window + 1, 1)
            y_batch = K.utils.to_categorical(y_batch, num_classes=6)
            yield X_batch, y_batch

Train/test split our data:

In [128]:
df_train, df_test = train_test_split(
        df_onehot,
        test_size=0.1,
        random_state=1337,
        shuffle=False)

In [129]:
df_train.head()

,orientation_is_N,orientation_is_S,orientation_is_E,orientation_is_W,parent_exit_is_0,parent_exit_is_1,parent_exit_is_2,parent_exit_is_3,parent_exit_is_4,parent_exit_is_5,...,room_is_FiveWayCrossing,room_is_ChestCorridor,room_is_Library,room_is_PortalRoom,room_is_SmallCorridor,room_is_Start,room_is_None,stronghold,downwards,exit
0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
1,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
2,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,2
4,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,1


In [130]:
print("df_train:", df_train.shape)
print("df_test:", df_test.shape)

df_train: (631518, 111)
df_test: (70169, 111)


In [154]:
n_features = df_train.shape[1] - 2

Build and compile the actual model: an LSTM followed by a dense layer with softmax activation for our 6 exit classes.

In [152]:
model = K.Sequential()
model.add(K.layers.LSTM(
        64,
        batch_input_shape=(None, None, n_features),
        return_sequences=False,
        stateful=False))
model.add(K.layers.Dense(
        6,
        activation='softmax'))
model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy'])
model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_12 (LSTM)               (None, 64)                44544     
_________________________________________________________________
dense_18 (Dense)             (None, 6)                 390       
Total params: 44,934
Trainable params: 44,934
Non-trainable params: 0
_________________________________________________________________


Fit the model…

In [153]:
model.fit(
        sequencelify(df_train),
        epochs=10,
        steps_per_epoch=10000)

Epoch 1/10
10000/10000 [==============================] - 11s 1ms/step - loss: 0.4599 - accuracy: 0.7935
Epoch 2/10
10000/10000 [==============================] - 10s 1ms/step - loss: 0.4187 - accuracy: 0.8033
Epoch 3/10
10000/10000 [==============================] - 10s 1ms/step - loss: 0.4151 - accuracy: 0.8069
Epoch 4/10
10000/10000 [==============================] - 10s 1ms/step - loss: 0.4094 - accuracy: 0.8106
Epoch 5/10
10000/10000 [==============================] - 10s 1ms/step - loss: 0.4070 - accuracy: 0.8121
Epoch 6/10
10000/10000 [==============================] - 10s 1ms/step - loss: 0.4071 - accuracy: 0.8128
Epoch 7/10
10000/10000 [==============================] - 10s 1ms/step - loss: 0.4034 - accuracy: 0.8132
Epoch 8/10
10000/10000 [==============================] - 10s 994us/step - loss: 0.4035 - accuracy: 0.8126
Epoch 9/10
10000/10000 [==============================] - 10s 1ms/step - loss: 0.4067 - accuracy: 0.8126
Epoch 10/10
10000/10000 [============================

Evaluate the model on test data…

In [155]:
model.evaluate(sequencelify(df_test))

90333/90333 [==============================] - 101s 1ms/step - loss: 0.2942 - accuracy: 0.8605


[0.2941755950450897, 0.8605442643165588]

Save/load model weights.

In [156]:
model.save("rnn_2.keras")
#model = K.models.load_model("rnn_2.keras")